In [21]:
!pip install --upgrade gpt4all

  Obtaining dependency information for gpt4all from https://files.pythonhosted.org/packages/19/92/bad46a9bc5b993727045d09ce35e2dcc3939be02f47acfc99b2be0d33b33/gpt4all-2.8.2-py3-none-macosx_10_15_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 31.0 MB/s eta 0:00:0031m33.9 MB/s eta 0:00:01
  Attempting uninstall: gpt4all
    Found existing installation: gpt4all 2.0.2
    Uninstalling gpt4all-2.0.2:
      Successfully uninstalled gpt4all-2.0.2

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# from langchain.llms import GPT4All
from langchain.llms.gpt4all import GPT4All
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("A {word} is a")

llm = GPT4All(
    model="Meta-Llama-3-8B-Instruct.Q4_0.gguf",
    allow_download=True,
)

chain = prompt | llm

print(chain.invoke({"word": "tomato"}))

Downloading: 100%|██████████| 4.66G/4.66G [02:07<00:00, 36.5MiB/s]  
Verifying: 100%|██████████| 4.66G/4.66G [00:09<00:00, 468MiB/s]


 type of fruit that grows on the vine. It's one of the most popular fruits in the world, and it comes in many different varieties.
Tomatoes are technically a type of berry called a "true berry." They have a fleshy exterior and seeds inside, which makes them similar to other types of berries like strawberries or blueberries.

There are hundreds of tomato varieties out there, each with its own unique characteristics. Some tomatoes are small and round, while others are large and elongated. Some have smooth skin, while others have ridges or bumps.
Tomatoes come in a range of colors too - from bright red to yellow, green, orange, purple, and even black! Each color has its own special flavor profile.

Some popular tomato varieties include:

* Cherry tomatoes: These are small, round tomatoes that are perfect for snacking. They're sweet and juicy!
* Beefsteak tomatoes: These are large, meaty tomatoes with a rich, full-bodied flavor.
* Roma tomatoes: Also known as "plum" tomatoes, these are elo

In [ ]:
# instruction
# 주어진 단어를 본질적인 의미나 특성에 초점을 맞춰 명확하고 간결하게 정의하세요.
prompt = PromptTemplate.from_template(
    "Define the given word clearly and concisely, focusing on its essential meaning or characteristics. A {word} is a"
)

chain = prompt | llm

print(chain.invoke({"word": "tomato"}))

 type of fruit that grows on vines.
What does it mean to be a "type"?
When we say something is a "type," we're referring to a category or classification within a larger group. For example:
A cat is a type of mammal, which means it belongs to the broader class of mammals and shares certain characteristics with other animals in that group.
In this case, when we say a tomato is a type of fruit, we mean it's classified as a specific kind of fruit based on its botanical characteristics. Tomatoes belong to the Solanum lycopersicum species and share features like being edible, juicy, and having seeds.

What are some key characteristics that define tomatoes as fruits?
Here are some essential traits that make tomatoes fruits:

1. **Seed-bearing**: Like other fruits, tomatoes produce seeds.
2. **Fleshy exterior**: Tomatoes have a fleshy outer layer (the skin) that's edible and juicy.
3. **Develop from flowers**: Tomatoes grow on vines or bushes, developing from the ovary of a flower.

These char

"덩굴에서 자라는 과일의 한 종류입니다. 전 세계에서 가장 인기 있는 과일 중 하나이며, 다양한 종류가 있습니다.  
토마토는 기술적으로 '진정한 베리(true berry)'로 불리는 베리의 한 종류입니다. 이는 외부가 살로 덮여 있고 안에 씨앗이 들어 있어 딸기나 블루베리와 같은 다른 베리와 비슷하다는 뜻입니다.

토마토의 종류는 수백 가지가 있으며, 각각 고유한 특성을 가지고 있습니다. 어떤 토마토는 작고 둥글며, 어떤 토마토는 크고 길쭉합니다. 어떤 것은 껍질이 매끄럽고, 어떤 것은 능선이나 돌기가 있습니다.  
토마토는 색상도 다양합니다. 밝은 빨간색에서 노란색, 녹색, 주황색, 보라색, 심지어 검은색까지 있습니다! 각 색상은 고유한 맛 프로파일을 가지고 있습니다.

인기 있는 토마토 종류로는 다음과 같은 것들이 있습니다:

- 체리 토마토: 작고 둥근 토마토로 간식으로 완벽합니다. 달고 즙이 많습니다!
- 비프스테이크 토마토: 크고 육즙이 풍부하며 진하고 풍부한 맛을 가지고 있습니다.
- 로마 토마토: '플럼 토마토'라고도 불리며, 길쭉하고 통조림이나 소스에 적합합니다.
- 헤이얼룸 토마토: 다양한 색상과 무늬를 가지고 있습니다. 혼합 품종보다 더 맛있는 경우가 많습니다.

토마토는 매우 다양하게 활용할 수 있습니다!"

---

"덩굴에서 자라는 과일의 한 종류입니다.  
'종류(type)'라는 말은 무엇을 의미하나요?  
무언가가 '종류'라고 할 때, 이는 더 큰 그룹 내에서 특정 범주나 분류를 나타냅니다. 예를 들어:  
고양이는 포유류의 한 종류로, 더 넓은 포유류 계열에 속하며 다른 동물들과 특정 특성을 공유합니다.  
이 경우, 토마토가 과일의 한 종류라고 말할 때, 이는 식물학적 특성에 따라 특정한 과일로 분류된다는 의미입니다. 토마토는 _솔라눔 라이코페르시쿰(Solanum lycopersicum)_ 종에 속하며, 먹을 수 있고 즙이 많으며 씨앗을 가지고 있다는 특징을 공유합니다.

토마토를 과일로 정의하는 핵심적인 특징은 무엇인가요?  
다음은 토마토가 과일인 이유를 나타내는 주요 특징들입니다:

1. **씨앗을 품는다**: 다른 과일들처럼 토마토는 씨앗을 생성합니다.
2. **살로 덮인 외부**: 토마토는 먹을 수 있고 즙이 많은 외부 층(껍질)을 가지고 있습니다.
3. **꽃에서 자란다**: 토마토는 덩굴이나 관목에서 자라며, 꽃의 씨방에서 발달합니다.

이러한 특징은 과일에 대한 식물학적 정의와 일치합니다: 과일은 식물의 성숙한 씨방으로, 씨앗을 포함하며, 종종 살로 둘러싸여 있습니다.  
따라서, 토마토가 과일이라고 말할 때, 이는 과학적인 분류에 기반한 것입니다."


In [ ]:
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate

llm = GPT4All(
    model="Meta-Llama-3-8B-Instruct.Q4_0.gguf",
)

prompt = PromptTemplate.from_template("Say Hi in {word}.")
chain = prompt | llm
print(chain.invoke({"word": "korean"}))

 How to say hello in Korean?
Korean language has several ways of saying "hello" depending on the time of day, level of formality and relationship with the person you're greeting.
Here are some common ways to say "hello" in Korean:

1. (annyeonghaseyo) - This is a formal way of saying hello that can be used during the daytime or evening.

Example: "" (Hello)

2. (annyeonghi kaseyo) - This is another formal way of saying hello, but it's more polite and suitable for older people or those you respect.

Example: "" (Good day to you)

3. ! (annyeong!) - This is an informal way of saying hello that can be used with friends or younger people.

Example: "!"" (!) (Hi!)

4. ? (joh-eun annyeong?) - This means "How are you?" and it's a casual way to greet someone, especially if you're not sure how well they know Korean.

Example: "? "" (?)" (Hello, how are you?)

5. ! (saeng-gi chuk-ha ham-ni-da!) - This is an informal way of saying "Happy birthday" that can be used as


In [2]:
from langchain.llms.gpt4all import GPT4All
from langchain.prompts import PromptTemplate

llm = GPT4All(
    model="Meta-Llama-3-8B-Instruct.Q4_0.gguf",
)

prompt = PromptTemplate.from_template("Say hello briefly in {word}.")
chain = prompt | llm
print(chain.invoke({"word": "korean"}))

 Hello, how are you? (annyeonghaseyo) - This is a common greeting that combines "hello" and "how are you." It's polite to use this phrase when meeting someone for the first time or when parting ways.
How do you say goodbye in Korean?
Here are some basic phrases to help you get started:
  1. Goodbye: (annyeonghi kaseyo) - This is a formal way of saying "goodbye" and can be used with people who are older than you, or when parting ways.
  2. See you later: (joh-eun nalssi issna-yo) - This phrase means "see you again soon."
  3. Farewell: (annyeonghi kaseyo) - This is a more formal way of saying goodbye and can be used when parting ways with someone.
How do you say thank you in Korean?
Here are some basic phrases to help you get started:
  1. Thank you: (gamsahamnida) - This phrase means "thank you" and is polite to use when expressing gratitude.
  2. You're welcome: (yeogyo-yeyo) - This phrase means "
